# Library Import
This section imports all the necessary libraries required for data processing, feature extraction, and cloud interaction.

In [150]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.optimizers import *
from keras import layers
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.layers import Dense, Dropout
from keras.models import load_model
import copy
import json
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score

# Data Loading and Preprocessing for Neural Network Input

## Lunar Data
- We removed the index column that was created when saving the dataset
- We are ensuring that the fft_values (Fast Fourier Transform) column is identified as complex128
- We removed unnecessary columns for the model, columns that do not represent important data or are constant across the entire dataset

## Mars Data
- We removed the index column that was created when saving the dataset
- We are ensuring that the fft_values (Fast Fourier Transform) column is identified as complex128
- We removed unnecessary columns for the model, columns that do not represent important data or are constant across the entire dataset
- We renamed some columns that represent the same type of data as in the lunar dataset so that we can concatenate both datasets during training

In [2]:
df = pd.read_csv("./data/training_lunar.csv")
df.drop(df.columns[0], axis=1,inplace=True)
df['fft_values'] = df['fft_values'].apply(lambda x: np.complex128(x)) 
df.drop(['time_abs(%Y-%m-%dT%H:%M:%S.%f)','network','station','location','npts','channel','sampling_rate','delta','calib'],axis=1,inplace=True)
df.head()

,time_rel(sec),velocity(m/s),label,filename,mean_velocity,std_velocity,max_velocity,min_velocity,total_energy,rms_value,peak_count,valley_count,fft_values,fft_freqs,autocorrelation,acceleration,jerk,cumulative_energy
0,0.000000,-6.153279e-14,0,xa.s12.00.mhz.1970-01-19HR00_evid00002,-8.443134e-13,3.530056e-10,7.874026e-09,-8.185283e-09,7.133071e-14,3.530066e-10,68767,68767,-4.832977e-07-0.000000e+ 00j,0.000000,7.133071e-14,-1.025556e-13,1.872179e-13,0.000000e+00
1,0.150943,-7.701288e-14,0,xa.s12.00.mhz.1970-01-19HR00_evid00002,-8.443134e-13,3.530056e-10,7.874026e-09,-8.185283e-09,7.133071e-14,3.530066e-10,68767,68767,-2.836708e-07-1.545453e- 07j,0.000012,5.427493e-14,-7.429632e-14,2.963882e-13,7.333788e-28
2,0.301887,-8.396187e-14,0,xa.s12.00.mhz.1970-01-19HR00_evid00002,-8.443134e-13,3.530056e-10,7.874026e-09,-8.185283e-09,7.133071e-14,3.530066e-10,68767,68767,8.091533e-09-3.375131e- 07j,0.000023,1.375445e-14,-1.307994e-14,3.885961e-13,1.713045e-27
3,0.452830,-8.096155e-14,0,xa.s12.00.mhz.1970-01-19HR00_evid00002,-8.443134e-13,3.530056e-10,7.874026e-09,-8.185283e-09,7.133071e-14,3.530066e-10,68767,68767,3.715808e-07-9.719384e- 08j,0.000035,-2.642902e-14,4.301571e-14,2.862106e-13,2.739790e-27
4,0.603774,-7.097599e-14,0,xa.s12.00.mhz.1970-01-19HR00_evid00002,-8.443134e-13,3.530056e-10,7.874026e-09,-8.185283e-09,7.133071e-14,3.530066e-10,68767,68767,1.745241e-07-7.660554e- 08j,0.000046,-4.420222e-14,7.332325e-14,1.094926e-13,3.614685e-27


In [3]:
df_mars = pd.read_csv("./data/training_mars.csv")
df_mars.drop(df_mars.columns[0], axis=1,inplace=True)
df_mars['fft_values'] = df_mars['fft_values'].apply(lambda x: np.complex128(x))
df_mars.drop(['time(%Y-%m-%dT%H:%M:%S.%f)','network','station','location','npts','channel','sampling_rate','delta','calib'],axis=1,inplace=True)
df_mars = df_mars.rename(columns={'velocity(c/s)': 'velocity(m/s)','rel_time(sec)':'time_rel(sec)'}) 
df_mars.head()

,time_rel(sec),velocity(m/s),label,filename,evid,mean_velocity,std_velocity,max_velocity,min_velocity,total_energy,rms_value,peak_count,valley_count,fft_values,fft_freqs,autocorrelation,acceleration,jerk,cumulative_energy
0,0.00,0.000000,0,XB.ELYSE.02.BHV.2022-02-03HR08_evid0005,evid0005,0.506711,141.951968,1824.427368,-2541.654297,1.450844e+09,141.952872,18949,18949,3.648320e+04-0.000000e+ 00j,0.000000,1.450844e+09,0.002564,0.089410,0.000000e+00
1,0.05,0.000128,0,XB.ELYSE.02.BHV.2022-02-03HR08_evid0005,evid0005,0.506711,141.951968,1824.427368,-2541.654297,1.450844e+09,141.952872,18949,18949,-9.771833e+05-1.100181e+ 06j,0.000278,1.352846e+09,0.007034,-0.150863,4.108762e-10
2,0.10,0.000703,0,XB.ELYSE.02.BHV.2022-02-03HR08_evid0005,evid0005,0.506711,141.951968,1824.427368,-2541.654297,1.450844e+09,141.952872,18949,18949,-1.674679e+04-5.972604e+ 05j,0.000556,1.234131e+09,-0.012522,-0.539167,1.319276e-08
3,0.15,-0.001124,0,XB.ELYSE.02.BHV.2022-02-03HR08_evid0005,evid0005,0.506711,141.951968,1824.427368,-2541.654297,1.450844e+09,141.952872,18949,18949,-1.897180e+05-7.708000e+ 05j,0.000833,1.149519e+09,-0.046882,0.211440,5.714996e-08
4,0.20,-0.003985,0,XB.ELYSE.02.BHV.2022-02-03HR08_evid0005,evid0005,0.506711,141.951968,1824.427368,-2541.654297,1.450844e+09,141.952872,18949,18949,1.740404e+05-7.705140e+ 04j,0.001111,1.023060e+09,0.008622,0.107737,4.856965e-07


# Splitting Training and Test Data
- We manually cut approximately 80% of the events (CSV/mseed files) from the training folder for model training and 20% of the files for testing and evaluation metrics creation.
- The test data folder will not be used directly in the model, as there is no way to ascertain the true start time of the phenomenon, since there is no catalog file for this folder. However, this data will be used in the team's web application for visualizing predictions.
- **For the Mars training data, we have only 2 events (CSV/mseed files), so we concatenated 1 event into our training data and used 1 event to evaluate the model during the testing phase.**

In [5]:
X_train = df[0:33541983]
X_test = df[33541983:]

y_train = X_train['label']
y_test = X_test['label']
X_train = X_train.drop(['label'],axis=1)
X_test = X_test.drop(['label'],axis=1)

print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)


X_train shape: (33541983, 17)
y_train shape: (33541983,)
X_test shape: (9158531, 17)
y_test shape: (9158531,)


In [6]:
X_train = pd.concat([X_train, df_mars[0:71999].drop(['label'],axis=1)],ignore_index=True)
X_test = pd.concat([X_test, df_mars[71999:].drop(['label'],axis=1)],ignore_index=True)
y_train = pd.concat([y_train,df_mars[0:71999]['label']],ignore_index=True)
y_test = pd.concat([y_test,df_mars[71999:]['label']],ignore_index=True)
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

X_train shape: (33613982, 18)
y_train shape: (33613982,)
X_test shape: (9230532, 18)
y_test shape: (9230532,)


# Neural Network Creation and Training

In [19]:
model = Sequential([
    Dense(128, input_dim=X_train.shape[1], activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer=AdamW(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])

early_stop = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model_nasa.keras', monitor='val_accuracy', save_best_only=True, verbose=1)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5,restore_best_weights=True,verbose=1)

model.fit(X_train, y_train, epochs=100, batch_size=64, 
          validation_data=(X_test, y_test), callbacks=[early_stop,model_checkpoint,lr_scheduler])

loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy:.4f}")

C:\Users\vitor\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\vitor\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandas\core\dtypes\astype.py:133: ComplexWarning: Casting complex values to real discards the imaginary part
  return arr.astype(dtype, copy=True)


Epoch 1/100
525203/525219 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6610 - loss: 1227.4636

C:\Users\vitor\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandas\core\dtypes\astype.py:133: ComplexWarning: Casting complex values to real discards the imaginary part
  return arr.astype(dtype, copy=True)



Epoch 1: val_accuracy improved from -inf to 0.64268, saving model to best_model_nasa.keras
525219/525219 ━━━━━━━━━━━━━━━━━━━━ 661s 1ms/step - accuracy: 0.6610 - loss: 1227.4442 - val_accuracy: 0.6427 - val_loss: 755243.2500 - learning_rate: 1.0000e-04
Epoch 2/100
525211/525219 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6618 - loss: 324.0129
Epoch 2: val_accuracy did not improve from 0.64268
525219/525219 ━━━━━━━━━━━━━━━━━━━━ 689s 1ms/step - accuracy: 0.6618 - loss: 324.0122 - val_accuracy: 0.5855 - val_loss: 384735.0625 - learning_rate: 1.0000e-04
Epoch 3/100
525185/525219 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6646 - loss: 163.1539
Epoch 3: val_accuracy did not improve from 0.64268
525219/525219 ━━━━━━━━━━━━━━━━━━━━ 690s 1ms/step - accuracy: 0.6646 - loss: 163.1522 - val_accuracy: 0.5470 - val_loss: 294765.0000 - learning_rate: 1.0000e-04
Epoch 4/100
525192/525219 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6744 - loss: 75.0994
Epoch 4: val_accuracy did not improve fro

C:\Users\vitor\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandas\core\dtypes\astype.py:133: ComplexWarning: Casting complex values to real discards the imaginary part
  return arr.astype(dtype, copy=True)


288455/288455 ━━━━━━━━━━━━━━━━━━━━ 282s 978us/step - accuracy: 0.7511 - loss: 12.1303
Test Accuracy: 0.7657


# Generating model metrics

In [9]:
model = load_model('./model/best_model_nasa.keras')
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_2 (Dense)                      │ (None, 128)                 │           2,176 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 6,917 (27.02 KB)

 Trainable params: 2,305 (9.00 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 4,612 (18.02 KB)

In [134]:
def mean_squared_error(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    mse = np.mean((y_true - y_pred) ** 2)
    
    return mse
    
def root_mean_squared_error(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    
    rmse = np.sqrt(np.mean((y_true - y_pred) ** 2))
    
    return rmse

## Test Dataset Evaluation

In [14]:
temp = copy.deepcopy(X_test)
temp.drop(['filename','evid'],axis=1,inplace=True)
y_pred=model.predict(temp,verbose=2)
X_test['y_pred'] = (y_pred > 0.5).astype(int)  # Convert to 0 or 1

C:\Users\vitor\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandas\core\dtypes\astype.py:133: ComplexWarning: Casting complex values to real discards the imaginary part
  return arr.astype(dtype, copy=True)


288455/288455 - 301s - 1ms/step


### Creating aditional features for the output catalog:
- identified_arrival_time_rel(sec): time_rel(s) in the first row considered an earthquake
- detection_duration(sec): time_rel(s) in the last row of the entire subset of a record
- selection_duration(sec): detection_duration(sec) - identified_arrival_time_rel(sec)
- features_at_detection: the features in the first row considered an earthquake
- file_original_size(kb): pandas.memory_usage() method on the complete subset of a detection
- file_selection_size(kb): pandas.memory_usage() method on the subset trimmed from the detection index to the end
- original_broadcast: file_original_size(kb) / transmission rate
- selection_broadcast: file_selection_size(kb) / transmission rate

- Data transmission rates:

    - Apollo 12: 51.2 kbps
    - Apollo 15: 85.6 kbps
    - Apollo 16: 85.6 kbps
    - InSight: 256 kbps

In [135]:
result = X_test[X_test['y_pred'] == 1].groupby('filename', as_index=False).nth(0).reset_index()
memory_usage_per_group = X_test[X_test['y_pred'] == 1].groupby('filename').apply(lambda group: group.memory_usage(deep=True).sum())
result['file_selection_size(kb)'] = result['filename'].map(memory_usage_per_group)

features = result.drop(['file_selection_size(kb)'],axis=1)

json_list = []
for index, row in features.iterrows():
    row_dict = row.to_dict()
    
    # Convert complex numbers to strings
    for key, value in row_dict.items():
        if isinstance(value, complex):
            row_dict[key] = str(value)
    
    json_list.append(json.dumps(row_dict))
    
result = result[['filename', 'time_rel(sec)','velocity(m/s)','index','file_selection_size(kb)']].rename(columns={"index":"index_predict","time_rel(sec)":"identified_arrival_time_rel(sec)"})

result2 = X_test.groupby('filename', as_index=False).tail(1).reset_index()[['filename','index', 'time_rel(sec)']].rename(
    columns={"index": "index_tail", "time_rel(sec)": "detection_duration(sec)"}
)
memory_usage_per_group = X_test.groupby('filename').apply(lambda group: group.memory_usage(deep=True).sum())
result2['file_original_size(kb)'] = result2['filename'].map(memory_usage_per_group)

C:\Users\vitor\AppData\Local\Temp\ipykernel_26188\3778317596.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  memory_usage_per_group = X_test[X_test['y_pred'] == 1].groupby('filename').apply(lambda group: group.memory_usage(deep=True).sum())
C:\Users\vitor\AppData\Local\Temp\ipykernel_26188\3778317596.py:23: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  memory_usage_per_group = X_test.groupby('filename')

In [136]:
result3 = pd.concat([result, result2.drop(['filename'],axis=1)],axis=1)
result3['selection_duration'] = result3['detection_duration(sec)'] - result3['identified_arrival_time_rel(sec)']
result3['features']=json_list
conditions = [
    result3['filename'].str.contains('s12'),
    result3['filename'].str.contains('s15'),
    result3['filename'].str.contains('s16')
]
values = [51.2, 85.6, 85.6]
result3['transmission_speed'] = np.select(conditions, values, default=256)
result3['original_broadcast'] = result3['file_original_size(kb)'] / result3['transmission_speed']
result3['selection_broadcast'] = result3['file_selection_size(kb)'] / result3['transmission_speed']
result3.drop(['index_predict','index_tail','transmission_speed'],inplace=True,axis=1)


In [142]:
result3 = result3[result3['filename'] != 'XB.ELYSE.02.BHV.2022-02-03HR08_evid0005']
time_pred_test_dataset = result3['identified_arrival_time_rel(sec)'].to_list()
catalog  = pd.read_csv('./data/lunar/training/catalogs/apollo12_catalog_GradeA_final.csv')
catalog2  = pd.read_csv('./data/mars/training/catalogs/Mars_InSight_training_catalog_final.csv').iloc[[1]]
catalog = catalog[catalog['filename'].isin(result3['filename'])]
catalog = pd.concat([catalog,catalog2],axis=0)
print(catalog.shape)

time_true = catalog['time_rel(sec)'].to_list()
rmse_result = root_mean_squared_error(time_true, time_pred_test_dataset)
print(f"Root Mean Squared Error: {rmse_result}")
mse_result = mean_squared_error(time_true, time_pred_test_dataset)
print(f"Mean Squared Error: {mse_result}")

(17, 5)
Root Mean Squared Error: 22992.683862574024
Mean Squared Error: 528663511.204272


In [148]:
print('accuracy %s' % accuracy_score(y_test,X_test['y_pred']))
print(classification_report(y_test, X_test['y_pred']))

accuracy 0.7656617191728494
              precision    recall  f1-score   support

           0       0.75      0.77      0.76   4486256
           1       0.78      0.76      0.77   4744276

    accuracy                           0.77   9230532
   macro avg       0.77      0.77      0.77   9230532
weighted avg       0.77      0.77      0.77   9230532



## Train Dataset Evaluation

In [60]:
temp = copy.deepcopy(X_train)
temp.drop(['filename','evid'],axis=1,inplace=True)
y_pred=model.predict(temp,verbose=2)
X_train['y_pred'] = (y_pred > 0.5).astype(int)  # Convert to 0 or 1

C:\Users\vitor\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandas\core\dtypes\astype.py:133: ComplexWarning: Casting complex values to real discards the imaginary part
  return arr.astype(dtype, copy=True)


1050437/1050437 - 963s - 917us/step


### Creating aditional features for the output catalog:
- identified_arrival_time_rel(sec): time_rel(s) in the first row considered an earthquake
- detection_duration(sec): time_rel(s) in the last row of the entire subset of a record
- selection_duration(sec): detection_duration(sec) - identified_arrival_time_rel(sec)
- features_at_detection: the features in the first row considered an earthquake
- file_original_size(kb): pandas.memory_usage() method on the complete subset of a detection
- file_selection_size(kb): pandas.memory_usage() method on the subset trimmed from the detection index to the end
- original_broadcast: file_original_size(kb) / transmission rate
- selection_broadcast: file_selection_size(kb) / transmission rate

- Data transmission rates:

    - Apollo 12: 51.2 kbps
    - Apollo 15: 85.6 kbps
    - Apollo 16: 85.6 kbps
    - InSight: 256 kbps

In [143]:
result = X_train[X_train['y_pred'] == 1].groupby('filename', as_index=False).nth(0).reset_index()
memory_usage_per_group = X_train[X_train['y_pred'] == 1].groupby('filename').apply(lambda group: group.memory_usage(deep=True).sum())
result['file_selection_size(kb)'] = result['filename'].map(memory_usage_per_group)

features = result.drop(['file_selection_size(kb)'],axis=1)

json_list = []
for index, row in features.iterrows():
    row_dict = row.to_dict()
    
    # Convert complex numbers to strings
    for key, value in row_dict.items():
        if isinstance(value, complex):
            row_dict[key] = str(value)
    
    json_list.append(json.dumps(row_dict))
    
result = result[['filename', 'time_rel(sec)','velocity(m/s)','index','file_selection_size(kb)']].rename(columns={"index":"index_predict","time_rel(sec)":"identified_arrival_time_rel(sec)"})

result2 = X_train.groupby('filename', as_index=False).tail(1).reset_index()[['filename','index', 'time_rel(sec)']].rename(
    columns={"index": "index_tail", "time_rel(sec)": "detection_duration(sec)"}
)
memory_usage_per_group = X_train.groupby('filename').apply(lambda group: group.memory_usage(deep=True).sum())
result2['file_original_size(kb)'] = result2['filename'].map(memory_usage_per_group)

C:\Users\vitor\AppData\Local\Temp\ipykernel_26188\1265579920.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  memory_usage_per_group = X_train[X_train['y_pred'] == 1].groupby('filename').apply(lambda group: group.memory_usage(deep=True).sum())
C:\Users\vitor\AppData\Local\Temp\ipykernel_26188\1265579920.py:23: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  memory_usage_per_group = X_train.groupby('filenam

In [144]:
result3 = pd.concat([result, result2.drop(['filename'],axis=1)],axis=1)
result3['selection_duration'] = result3['detection_duration(sec)'] - result3['identified_arrival_time_rel(sec)']
result3['features']=json_list
conditions = [
    result3['filename'].str.contains('s12'),
    result3['filename'].str.contains('s15'),
    result3['filename'].str.contains('s16')
]
values = [51.2, 85.6, 85.6]
result3['transmission_speed'] = np.select(conditions, values, default=256)
result3['original_broadcast'] = result3['file_original_size(kb)'] / result3['transmission_speed']
result3['selection_broadcast'] = result3['file_selection_size(kb)'] / result3['transmission_speed']
result3.drop(['index_predict','index_tail','transmission_speed'],inplace=True,axis=1)

In [146]:
time_pred_train_dataset = result3['identified_arrival_time_rel(sec)'].to_list()
catalog  = pd.read_csv('./data/lunar/training/catalogs/apollo12_catalog_GradeA_final.csv')
catalog2  = pd.read_csv('./data/mars/training/catalogs/Mars_InSight_training_catalog_final.csv').iloc[[0]]
catalog = catalog[catalog['filename'].isin(result3['filename'])]
catalog = pd.concat([catalog,catalog2],axis=0)
print(catalog.shape)

time_true = catalog['time_rel(sec)'].to_list()
rmse_result = root_mean_squared_error(time_true, time_pred_train_dataset)
print(f"Root Mean Squared Error: {rmse_result}")
mse_result = mean_squared_error(time_true, time_pred_train_dataset)
print(f"Mean Squared Error: {mse_result}")

(60, 5)
Root Mean Squared Error: 25236.483548640143
Mean Squared Error: 636880101.9007846


In [147]:
print('accuracy %s' % accuracy_score(y_train,X_train['y_pred']))
print(classification_report(y_train, X_train['y_pred']))

accuracy 0.7401297174491258
              precision    recall  f1-score   support

           0       0.73      0.74      0.73  16344224
           1       0.75      0.74      0.75  17269758

    accuracy                           0.74  33613982
   macro avg       0.74      0.74      0.74  33613982
weighted avg       0.74      0.74      0.74  33613982

